In [1]:
import requests
import pandas as pd 
from bs4 import BeautifulSoup
import string
import spacy
import re

In [2]:
url = """https://insights.blackcoffer.com/ai-tool-alexa-google-assistant-finance-banking-tool-future/"""

In [3]:
headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:60.0) Gecko/20100101 Firefox/60.0"}
page = requests.get(url, headers=headers)
soup = BeautifulSoup(page.content, 'html.parser')
soup=BeautifulSoup(page.content, 'html.parser')
title=soup.find('h1',class_="entry-title")
title=title.text.replace('\n'," ")
title

'How machine learning used in finance and banking?'

In [4]:
content=soup.findAll(attrs={'class':'td-post-content'})
content=content[0].text.replace('\n'," ")
content

' Through AI tools like natural language processing, Alexa and google assistant has led the retail industry in its rise towards conversational commerce. As if a customer was interacting with a clerk in a retail store, conversational commerce makes it possible for users to engage with software to research, purchase, or get customer assistance with products and services across a wide range of industries. With Alexa, for example, users can\xa0ask any Alexa-enabled device\xa0to add an item to an Amazon shopping cart, set a purchasing reminder when a product is running low, or carry out a complete purchase without having to access a shopping cart. The result is a seamless conversational experience that enables consumers to carry out transactions as quickly as it takes to speak a sentence. Through AI tools like natural language processing, Alexa has led the retail industry in its rise towards conversational commerce. As if a customer was interacting with a clerk in a retail store, conversati

In [5]:
#Punctuation
content = content.translate(str.maketrans('', '', string.punctuation)) 
content

' Through AI tools like natural language processing Alexa and google assistant has led the retail industry in its rise towards conversational commerce As if a customer was interacting with a clerk in a retail store conversational commerce makes it possible for users to engage with software to research purchase or get customer assistance with products and services across a wide range of industries With Alexa for example users can\xa0ask any Alexaenabled device\xa0to add an item to an Amazon shopping cart set a purchasing reminder when a product is running low or carry out a complete purchase without having to access a shopping cart The result is a seamless conversational experience that enables consumers to carry out transactions as quickly as it takes to speak a sentence Through AI tools like natural language processing Alexa has led the retail industry in its rise towards conversational commerce As if a customer was interacting with a clerk in a retail store conversational commerce ma

In [6]:
text = content.split()
text


['Through',
 'AI',
 'tools',
 'like',
 'natural',
 'language',
 'processing',
 'Alexa',
 'and',
 'google',
 'assistant',
 'has',
 'led',
 'the',
 'retail',
 'industry',
 'in',
 'its',
 'rise',
 'towards',
 'conversational',
 'commerce',
 'As',
 'if',
 'a',
 'customer',
 'was',
 'interacting',
 'with',
 'a',
 'clerk',
 'in',
 'a',
 'retail',
 'store',
 'conversational',
 'commerce',
 'makes',
 'it',
 'possible',
 'for',
 'users',
 'to',
 'engage',
 'with',
 'software',
 'to',
 'research',
 'purchase',
 'or',
 'get',
 'customer',
 'assistance',
 'with',
 'products',
 'and',
 'services',
 'across',
 'a',
 'wide',
 'range',
 'of',
 'industries',
 'With',
 'Alexa',
 'for',
 'example',
 'users',
 'can',
 'ask',
 'any',
 'Alexaenabled',
 'device',
 'to',
 'add',
 'an',
 'item',
 'to',
 'an',
 'Amazon',
 'shopping',
 'cart',
 'set',
 'a',
 'purchasing',
 'reminder',
 'when',
 'a',
 'product',
 'is',
 'running',
 'low',
 'or',
 'carry',
 'out',
 'a',
 'complete',
 'purchase',
 'without',
 'havi

In [7]:
len(text)

504

In [8]:
#Positive Score 
with open(r"C:\Users\Om Bhandwalkar\Desktop\pos\positive-words.txt") as pos:
    poswords = pos.read().split("\n")  
    poswords = poswords[5:]
pos_count = " ".join ([w for w in text if w in poswords])
pos_count=pos_count.split(" ")
Positive_score=len(pos_count)
print(Positive_score)

28


In [9]:
#Negative Score
with open(r"C:\Users\Om Bhandwalkar\Desktop\pos\negative-words.txt",encoding ="ISO-8859-1") as neg:
    negwords = neg.read().split("\n")
    
negwords = negwords[36:]
neg_count = " ".join ([w for w in text if w in negwords])
neg_count=neg_count.split(" ")
Negative_score=len(neg_count)
print(Negative_score)

1


In [10]:
filter_content = ' '.join(text)
data=[[url,title,content,filter_content,Positive_score,Negative_score]]
data=pd.DataFrame(data,columns=["url","title","content","filter_content","Positive_Score","Negative_Score"])
from textblob import TextBlob
# Get The Subjectivity
def sentiment_analysis(data):
    sentiment = TextBlob(data["content"]).sentiment
    return pd.Series([sentiment.polarity,sentiment.subjectivity ])

# Adding Subjectivity & Polarity
data[["polarity", "subjectivity"]] = data.apply(sentiment_analysis, axis=1)

data

,url,title,content,filter_content,Positive_Score,Negative_Score,polarity,subjectivity
0,https://insights.blackcoffer.com/ai-tool-alexa...,How machine learning used in finance and banking?,Through AI tools like natural language proces...,Through AI tools like natural language process...,28,1,0.176131,0.531168


In [11]:
#AVG SENTENCE LENGTH
AVG_SENTENCE_LENGTH = len(content.replace(' ',''))/len(re.split(r'[?!.]', content))
print('Word average =', AVG_SENTENCE_LENGTH)
import textstat
#Fog index 
FOG_INDEX=(textstat.gunning_fog(content))
print("FOG INDEX = ",FOG_INDEX)
#Average No of Words Per Sentence 
AVG_NUMBER_OF_WORDS_PER_SENTENCE = [len(l.split()) for l in re.split(r'[?!.]', content) if l.strip()]
print("Average no of words per sentence")
AVG_NUMBER_OF_WORDS_PER_SENTENCE=print(sum(AVG_NUMBER_OF_WORDS_PER_SENTENCE)/len(AVG_NUMBER_OF_WORDS_PER_SENTENCE))
#Complex words
def syllable_count(word):
    count = 0
    vowels = "AEIOUYaeiouy"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)): 
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
            if word.endswith("es"or "ed"):
                count -= 1
    if count == 0:
        count += 1
    return count



COMPLEX_WORDS=syllable_count(content)
print("Complex Words",COMPLEX_WORDS)


Word average = 2746.0
FOG INDEX =  206.76
Average no of words per sentence
504.0
Complex Words 1003


In [12]:
#Word Count
Word_Count=len(content)
print("Word Count",Word_Count)
#Percentage Complex Words
pcw=(COMPLEX_WORDS/Word_Count)*100
print("Percentage of Complex Words",pcw)
#Average Word Length
Average_Word_Length=len(content.replace(' ',''))/len(content.split())
print("Average Word per Length",Average_Word_Length)
#Syllable Count Per Word
word=content.replace(' ','')
syllable_count=0
for w in word:
      if(w=='a' or w=='e' or w=='i' or w=='o' or w=='y' or w=='u' or w=='A' or w=='E' or w=='I' or w=='O' or w=='U' or w=='Y'):
            syllable_count=syllable_count+1
print("The AVG number of syllables in the word is: ")
print(syllable_count/len(content.split()))

Word Count 3246
Percentage of Complex Words 30.899568699938385
Average Word per Length 5.448412698412699
The AVG number of syllables in the word is: 
2.2023809523809526


In [1]:
import requests
import pandas as pd 
from bs4 import BeautifulSoup
import string
import spacy
import re
url = """https://insights.blackcoffer.com/ai-tool-alexa-google-assistant-finance-banking-tool-future/"""
headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:60.0) Gecko/20100101 Firefox/60.0"}
page = requests.get(url, headers=headers)
soup = BeautifulSoup(page.content, 'html.parser')
soup=BeautifulSoup(page.content, 'html.parser')
title=soup.find('h1',class_="entry-title")
title=title.text.replace('\n'," ")
# title

content=soup.findAll(attrs={'class':'td-post-content'})
content=content[0].text.replace('\n'," ")
# print(content)
#Punctuation
content = content.translate(str.maketrans('', '', string.punctuation)) 
# print(content)
text = content.split()
# print(text)
len(text)
#Positive Score 
with open(r"C:\Users\Om Bhandwalkar\Desktop\pos\positive-words.txt") as pos:
    poswords = pos.read().split("\n")  
    poswords = poswords[5:]
pos_count = " ".join ([w for w in text if w in poswords])
pos_count=pos_count.split(" ")
Positive_score=len(pos_count)
print(Positive_score)

#Negative Score
with open(r"C:\Users\Om Bhandwalkar\Desktop\pos\negative-words.txt",encoding ="ISO-8859-1") as neg:
    negwords = neg.read().split("\n")
    
negwords = negwords[36:]
neg_count = " ".join ([w for w in text if w in negwords])
neg_count=neg_count.split(" ")
Negative_score=len(neg_count)
print(Negative_score)


filter_content = ' '.join(text)
data=[[url,title,content,filter_content,Positive_score,Negative_score]]
data=pd.DataFrame(data,columns=["url","title","content","filter_content","Positive_Score","Negative_Score"])
from textblob import TextBlob
# Get The Subjectivity
def sentiment_analysis(data):
    sentiment = TextBlob(data["content"]).sentiment
    return pd.Series([sentiment.polarity,sentiment.subjectivity ])
data[["polarity", "subjectivity"]] = data.apply(sentiment_analysis, axis=1)
data
#AVG SENTENCE LENGTH
AVG_SENTENCE_LENGTH = len(content.replace(' ',''))/len(re.split(r'[?!.]', content))
print('Word average =', AVG_SENTENCE_LENGTH)
import textstat
#Fog index 
FOG_INDEX=(textstat.gunning_fog(content))
print("FOG INDEX = ",FOG_INDEX)
#Average No of Words Per Sentence 
AVG_NUMBER_OF_WORDS_PER_SENTENCE = [len(l.split()) for l in re.split(r'[?!.]', content) if l.strip()]
print("Average no of words per sentence")
AVG_NUMBER_OF_WORDS_PER_SENTENCE=(sum(AVG_NUMBER_OF_WORDS_PER_SENTENCE)/len(AVG_NUMBER_OF_WORDS_PER_SENTENCE))
print(AVG_NUMBER_OF_WORDS_PER_SENTENCE)
#Complex words
def syllable_count(word):
    count = 0
    vowels = "AEIOUYaeiouy"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)): 
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
            if word.endswith("es"or "ed"):
                count -= 1
    if count == 0:
        count += 1
    return count



COMPLEX_WORDS=syllable_count(content)
print("Complex Words",COMPLEX_WORDS)
#Word Count
Word_Count=len(content)
print("Word Count",Word_Count)
#Percentage Complex Words
pcw=(COMPLEX_WORDS/Word_Count)*100
print("Percentage of Complex Words",pcw)
#Average Word Length
Average_Word_Length=len(content.replace(' ',''))/len(content.split())
print("Average Word per Length",Average_Word_Length)
#Syllable Count Per Word
word=content.replace(' ','')
syllable_count=0
for w in word:
      if(w=='a' or w=='e' or w=='i' or w=='o' or w=='y' or w=='u' or w=='A' or w=='E' or w=='I' or w=='O' or w=='U' or w=='Y'):
            syllable_count=syllable_count+1
print("The AVG number of syllables in the word is: ")
syllable = (syllable_count/len(content.split()))
print(syllable)

data = [[url,title,content,filter_content,Positive_score,Negative_score,AVG_SENTENCE_LENGTH,pcw,FOG_INDEX,
         AVG_NUMBER_OF_WORDS_PER_SENTENCE,COMPLEX_WORDS,Word_Count,syllable,Average_Word_Length]]
data=pd.DataFrame(data,columns=["url","title","content","filter_content","Positive_Score","Negative_Score","Avg_Sentence_Length"
                               ,"Percentage_Complex_Word","Fog_Index"," AVG_NUMBER_OF_WORDS_PER_SENTENCE","COMPLEX_WORDS",
                               "Word_Count","syllable","Average_Word_Length"])
from textblob import TextBlob
# Get The Subjectivity
def sentiment_analysis(data):
    sentiment = TextBlob(data["content"]).sentiment
    return pd.Series([sentiment.polarity,sentiment.subjectivity ])
data[["polarity", "subjectivity"]] = data.apply(sentiment_analysis, axis=1)
data



28
1
Word average = 2746.0
FOG INDEX =  206.76
Average no of words per sentence
504.0
Complex Words 1003
Word Count 3246
Percentage of Complex Words 30.899568699938385
Average Word per Length 5.448412698412699
The AVG number of syllables in the word is: 
2.2023809523809526


,url,title,content,filter_content,Positive_Score,Negative_Score,Avg_Sentence_Length,Percentage_Complex_Word,Fog_Index,AVG_NUMBER_OF_WORDS_PER_SENTENCE,COMPLEX_WORDS,Word_Count,syllable,Average_Word_Length,polarity,subjectivity
0,https://insights.blackcoffer.com/ai-tool-alexa...,How machine learning used in finance and banking?,Through AI tools like natural language proces...,Through AI tools like natural language process...,28,1,2746.0,30.899569,206.76,504.0,1003,3246,2.202381,5.448413,0.176131,0.531168


In [3]:
data.to_csv(r'C:\Users\Om Bhandwalkar\Desktop\BlackCoffer Assignment\Output\url_52.csv')